In [14]:
import pandas as pd
import numpy as np
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import glob

# Grouping scraped datasets

In [15]:
path = '../scraping/datasets_scraped'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0)
    li.append(frame)

#concatenating all datasets and removing duplicate rows (there might be duplicate posts)
df = pd.concat(li, axis=0, ignore_index=True)
df.drop_duplicates(inplace = True)

df.head()

,title,address,area,bathrooms,bedrooms,parking_spots,extra_contents,rent,fee
0,"Apartamento com 3 Quartos para Aluguel, 228m²","Rua dos Navegantes, 215 - Boa Viagem, Recife - PE",228,4,3,1,NaN,6.0,NaN
1,"Apartamento com 4 Quartos para Aluguel, 112m²","Rua Baltazar Passos, 260 - Boa Viagem, Recife ...",112,4,4,2,"['Piscina', 'Elevador', 'Academia', 'Churrasqu...",3.5,NaN
2,"Apartamento com 4 Quartos para Aluguel, 160m²","Avenida Boa Viagem, 5822 - Boa Viagem, Recife ...",160,5,4,3,"['Piscina', 'Elevador', 'Academia', 'Condomíni...",5.0,R$ 1.232
3,"Apartamento com 3 Quartos para Aluguel, 75m²","Tamarineira, Recife - PE",75,3,3,2,"['Piscina', 'Elevador', 'Academia', 'Playgroun...",3.2,NaN
4,"Apartamento com 5 Quartos para Aluguel, 200m²","Rua Setúbal, 278 - Boa Viagem, Recife - PE",200,5,5,3,"['Elevador', 'Ar-condicionado', 'Salão de fest...",6.0,NaN


In [16]:
df.shape

(3252, 9)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3252 entries, 0 to 6294
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3252 non-null   object 
 1   address         3252 non-null   object 
 2   area            3252 non-null   int64  
 3   bathrooms       3252 non-null   object 
 4   bedrooms        3252 non-null   int64  
 5   parking_spots   3252 non-null   object 
 6   extra_contents  2774 non-null   object 
 7   rent            3252 non-null   float64
 8   fee             1049 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 254.1+ KB


# Data cleaning

In [18]:
#checking null values in columns
df.isna().sum()

title                0
address              0
area                 0
bathrooms            0
bedrooms             0
parking_spots        0
extra_contents     478
rent                 0
fee               2203
dtype: int64

Here we'll have two different approaches.

- For the *extra_contents* column we've already filled the null values with a empty list. That way in the EDA we'll be able to turn those lists into columns. 

- For the *fee* column we'll fill the NAs with 0, most of the posts that do not contain a fee description also say that the value is already contained in rent.

In [19]:
#removing currency simbols, filling NAs with 0 and turning column into integer
df['fee'] = df['fee'].str.replace('.','').str.replace('$','').str.replace('R','').str.strip().astype(float)
df['fee'] = df.fee.fillna(0)
df['fee'] = df.fee.astype(int)

#turning into integers columns and replacing strings
df['bathrooms'] = df.bathrooms.str.replace('--', '0')
df['bathrooms'] = df.bathrooms.astype(str).astype(int)
df['parking_spots'] = df.parking_spots.str.replace('--', '0')
df['parking_spots'] = df.parking_spots.astype(str).astype(int)

#filling nas with a empty list and turning string list into list
df.extra_contents.fillna('[]', inplace = True)
df["extra_contents"] = df["extra_contents"].apply(eval)

#correcting the decimal places in the rent column and turning it into integer
df['rent'] = df.rent * 1000
df['rent'] = df.rent.astype(int)

C:\Users\arthu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


In [20]:
df.head()

,title,address,area,bathrooms,bedrooms,parking_spots,extra_contents,rent,fee
0,"Apartamento com 3 Quartos para Aluguel, 228m²","Rua dos Navegantes, 215 - Boa Viagem, Recife - PE",228,4,3,1,[],6000,0
1,"Apartamento com 4 Quartos para Aluguel, 112m²","Rua Baltazar Passos, 260 - Boa Viagem, Recife ...",112,4,4,2,"[Piscina, Elevador, Academia, Churrasqueira, P...",3500,0
2,"Apartamento com 4 Quartos para Aluguel, 160m²","Avenida Boa Viagem, 5822 - Boa Viagem, Recife ...",160,5,4,3,"[Piscina, Elevador, Academia, Condomínio fecha...",5000,1232
3,"Apartamento com 3 Quartos para Aluguel, 75m²","Tamarineira, Recife - PE",75,3,3,2,"[Piscina, Elevador, Academia, Playground, Salã...",3200,0
4,"Apartamento com 5 Quartos para Aluguel, 200m²","Rua Setúbal, 278 - Boa Viagem, Recife - PE",200,5,5,3,"[Elevador, Ar-condicionado, Salão de festas, A...",6000,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3252 entries, 0 to 6294
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           3252 non-null   object
 1   address         3252 non-null   object
 2   area            3252 non-null   int64 
 3   bathrooms       3252 non-null   int32 
 4   bedrooms        3252 non-null   int64 
 5   parking_spots   3252 non-null   int32 
 6   extra_contents  3252 non-null   object
 7   rent            3252 non-null   int32 
 8   fee             3252 non-null   int32 
dtypes: int32(4), int64(2), object(3)
memory usage: 203.2+ KB


In [22]:
df.isna().sum()

title             0
address           0
area              0
bathrooms         0
bedrooms          0
parking_spots     0
extra_contents    0
rent              0
fee               0
dtype: int64

In [23]:
df.title.str.split(' ', 1).str[0].unique()

array(['Apartamento'], dtype=object)

Here we can also see that all the properties have been classified as apartments.

# Feature engineering

In [24]:
#df['address'] = df.address.str.rsplit("-", 1).str[0]
df['address'] = df.address.str.strip()

In [25]:
locator = Nominatim(user_agent = 'myGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [26]:
#getting location data from geocode
df['location'] = df['address'].apply(locator.geocode)

GeocoderTimedOut: Service timed out

In [ ]:
df.head()

In [ ]:
df.isna().sum()

A few posts of our data could not have their location identified by geopy, since we'll need latitude and longitude for our future model, we'll remove those posts.

In [ ]:
#filtering out NAs in location column
df = df[df.location.notna()]

In [ ]:
#getting latitude and longitude
df['neighborhood'] = df.location.raw["address"]["suburb"]
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

#dropping unnecessary columns
df.drop(columns = ['location', 'point', 'altitude', 'title', 'address'], inplace = True)

Note that this project is built using Brazilian real estate market data, wich means that neighborhood is an important information when it comes to addresses. Also we took advantage of the geocoding to get the neighborhood column since our address column had some imperfections that would only make it harder to extract the variable.

In [ ]:
df.head()

In [ ]:
#taking values inside lists on "extra_contents" and turning into dummy variables
#code found on https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173

def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)


#creating dummy columns for each variable inside the extra_contents column            
extra_contents_df = boolean_df(df.extra_contents, to_1D(df.extra_contents)).astype(int)

#using pandas concat to add the new column to our dataset
df = pd.concat([df, extra_contents_df], axis=1)
df.drop(columns = ['extra_contents'], inplace = True)

In [ ]:
df.columns

We've now added 58 dummy columns to our dataset with extra contents about the apartments that will help us understand the data.

In [ ]:
#summing fee and rent column
df['rent'] = df['rent'] + df['fee']
df.drop(columns = ['fee'], inplace = True)

Most of the cases where the fee column is 0 is because this value is already included in the rent column. That's why we've summed up both columns.